In [ ]:
import sys
import time

import numpy as np

sys.path.append("../")
import warnings

import numpy as np
import torch
import torch.nn.functional as F
import utils
from gcn_models import GIN
from sklearn.metrics import accuracy_score, f1_score
from torch_geometric.loader import DataLoader

warnings.filterwarnings("ignore")

In [ ]:
start = time.time()

train_dataset = utils.GraphDataset("../data/", "GunPoint", True, True, n_quantiles=40)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

test_dataset = utils.GraphDataset("../data/", "GunPoint", False, True, n_quantiles=40)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

for elem in train_dataset:
    break
elem

In [ ]:
model = GIN(13, [10, 10, 10], out_channels=2, num_layers=1, pool_method="avg")

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        y_out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = F.cross_entropy(y_out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader, return_loss=True):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        y_out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y)
        if return_loss:
            loss += float(F.cross_entropy(y_out, data.y)) * data.num_graphs
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    if return_loss:
        return (
            f1_score(y_true=y_true, y_pred=y_pred, average="macro"),
            accuracy_score(y_true=y_true, y_pred=y_pred),
            loss / len(loader.dataset),
        )
    return f1_score(y_true=y_true, y_pred=y_pred, average="macro"), accuracy_score(
        y_true=y_true, y_pred=y_pred
    )

In [ ]:
best_macro_f1 = -1
train_losses = []
val_losses = []
train_accs = []
val_accs = []

In [ ]:
for epoch in range(2300):
    train()
    train_macro_f1, train_acc, train_loss = test(train_loader)
    test_macro_f1, test_acc, test_loss = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f},Test_Loss: {test_loss:02.4f},Train_f1: {train_macro_f1:01.4f},Test_f1: {test_macro_f1:01.4f},Train_acc: {train_acc:01.4f},Test_acc: {test_acc:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/quantile-PoC_GCNs.pth")
    train_losses.append(train_loss)
    val_losses.append(test_loss)
    train_accs.append(train_acc)
    val_accs.append(test_acc)
elapsed_time = time.time() - start
print(elapsed_time)

In [ ]:
utils.save_model_stats(
    "quantile-PoC_GCNs", train_losses, val_losses, train_accs, val_accs, elapsed_time
)